In [207]:
import numpy as np

In [208]:
%%script false --no-raise-error
# Above script disables this block

# Quad X
m1 = [-1,1,1,1]
m2 = [1,-1,1,1]
m3 = [1,1,-1,1]
m4 = [-1,-1,-1,1]
motors = [m1,m2,m3,m4]

Couldn't find program: 'false'


In [209]:
# %%script false --no-raise-error
# Above script disables this block

# Hexa X
m1 = [-1, 0, -1, 1]
m2 = [1, 0, 1, 1]
m3 = [0.5,-0.866,-1, 1]
m4 = [-0.5,0.866,1, 1]
m5 = [-0.5,-0.866,1, 1]
m6 = [0.5,0.866,-1, 1]
motors = [m1,m2,m3,m4,m5,m6]

In [210]:
EA = []
for motor in motors:
    # 0 - roll, 1 - pitch, 2 - yaw, 3 - thrust
    m = [motor[0], motor[1], motor[2], motor[3]]
    EA.append(m)
CA = np.linalg.pinv(EA)

print(f"Actuator Effectiveness Matrix: \n{EA}")
print(f"Control Allocation Matrix: \n{CA}")

Actuator Effectiveness Matrix: 
[[-1, 0, -1, 1], [1, 0, 1, 1], [0.5, -0.866, -1, 1], [-0.5, 0.866, 1, 1], [-0.5, -0.866, 1, 1], [0.5, 0.866, -1, 1]]
Control Allocation Matrix: 
[[-0.33333333  0.33333333  0.16666667 -0.16666667 -0.16666667  0.16666667]
 [ 0.          0.         -0.2886836   0.2886836  -0.2886836   0.2886836 ]
 [-0.16666667  0.16666667 -0.16666667  0.16666667  0.16666667 -0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]]


In [211]:
# Control allocation to Torques
# [Tp, Tq, Tr, T] = [CA] * [u1, u2, u3, u4, u5, u6]
Tp = Tq = Tr = 0
T = 10
Torq = [Tp, Tq, Tr, T]
u_input = [0, 0, 0, 0, 0, 0]

In [212]:
CA_inv = np.linalg.pinv(CA)
print(f"Inverted CA matrix: {CA_inv}")

Inverted CA matrix: [[-1.00000000e+00 -1.12940496e-16 -1.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  3.44445297e-16  1.00000000e+00  1.00000000e+00]
 [ 5.00000000e-01 -8.66000000e-01 -1.00000000e+00  1.00000000e+00]
 [-5.00000000e-01  8.66000000e-01  1.00000000e+00  1.00000000e+00]
 [-5.00000000e-01 -8.66000000e-01  1.00000000e+00  1.00000000e+00]
 [ 5.00000000e-01  8.66000000e-01 -1.00000000e+00  1.00000000e+00]]


In [213]:
# Compute u1,... u6 for equilibrium with some thrust T
# u_input = CA_inv * Torq
u_input = np.matmul(CA_inv, Torq)
print("At equilibrium")
print(f"u1, u2, u3, u4, u5, u6: {u_input}")

At equilibrium
u1, u2, u3, u4, u5, u6: [10. 10. 10. 10. 10. 10.]


In [214]:
# Remove 1 motor/ disable 1 motor
CA_fault_1 = []
CA_fault_1 = CA
CA_fault_1[:, 0] =  0
# Reset CA
CA = np.linalg.pinv(EA)
print(f"Control Allocation with 1 motor fault in m1: \n{CA_fault_1}")

Control Allocation with 1 motor fault in m1: 
[[ 0.          0.33333333  0.16666667 -0.16666667 -0.16666667  0.16666667]
 [ 0.          0.         -0.2886836   0.2886836  -0.2886836   0.2886836 ]
 [ 0.          0.16666667 -0.16666667  0.16666667  0.16666667 -0.16666667]
 [ 0.          0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]]


In [215]:
# Remove motor 3
print(f"CA: {CA}")
CA_fault_3 = CA
CA_fault_3[:, 2] =  0
# Reset CA
CA = np.linalg.pinv(EA)
print(f"Control Allocation with 1 motor fault in m3: \n{CA_fault_3}")

CA: [[-0.33333333  0.33333333  0.16666667 -0.16666667 -0.16666667  0.16666667]
 [ 0.          0.         -0.2886836   0.2886836  -0.2886836   0.2886836 ]
 [-0.16666667  0.16666667 -0.16666667  0.16666667  0.16666667 -0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]]
Control Allocation with 1 motor fault in m3: 
[[-0.33333333  0.33333333  0.         -0.16666667 -0.16666667  0.16666667]
 [ 0.          0.          0.          0.2886836  -0.2886836   0.2886836 ]
 [-0.16666667  0.16666667  0.          0.16666667  0.16666667 -0.16666667]
 [ 0.16666667  0.16666667  0.          0.16666667  0.16666667  0.16666667]]


In [216]:
# Use u1,...u6 values from equilibrium for finding Torque values instead
# [Tp, Tq, Tr, T] = [CA_fault] * [u1, u2, u3, u4, u5, u6]
# Torq_fault = CA_fault * u_input
Torq_fault = np.matmul(CA, u_input)
print(f"Control Allocation with NO FAULTs")
print(f"u1, u2, u3, u4, u5, u6: {u_input}")
print(f"Torque and thrust: {Torq_fault}")

Control Allocation with NO FAULTs
u1, u2, u3, u4, u5, u6: [10. 10. 10. 10. 10. 10.]
Torque and thrust: [-3.55271368e-15  3.55271368e-15 -3.33066907e-15  1.00000000e+01]


In [217]:
# If PWM input is NOT considering fault
u_input_fault =  [10, 10, 10, 10, 10, 10]
Torq_fault = np.matmul(CA_fault_1, u_input_fault)
print(f"\nControl Allocation with FAULT in m1")
print(f"If PWM input is NOT considering fault")
print(f"u1, u2, u3, u4, u5, u6: {u_input_fault}")
print(f"Torque and thrust: {Torq_fault}")

# If PWM input is considering fault
u_input_fault =  [0, 10, 10, 10, 10, 10]
Torq_fault = np.matmul(CA_fault_1, u_input_fault)
print(f"\nIf PWM input is considering fault")
print(f"u1, u2, u3, u4, u5, u6: {u_input_fault}")
print(f"Torque and thrust: {Torq_fault}")


Control Allocation with FAULT in m1
If PWM input is NOT considering fault
u1, u2, u3, u4, u5, u6: [10, 10, 10, 10, 10, 10]
Torque and thrust: [3.33333333 0.         1.66666667 8.33333333]

If PWM input is considering fault
u1, u2, u3, u4, u5, u6: [0, 10, 10, 10, 10, 10]
Torque and thrust: [3.33333333 0.         1.66666667 8.33333333]


In [218]:
# If PWM input is NOT considering fault
u_input_fault =  [10, 10, 10, 10, 10, 10]
Torq_fault = np.matmul(CA_fault_3, u_input_fault)
print(f"\nControl Allocation with FAULT in m3")
print(f"If PWM input is NOT considering fault")
print(f"u1, u2, u3, u4, u5, u6: {u_input_fault}")
print(f"Torque and thrust: {Torq_fault}")

# If PWM input is considering fault
u_input_fault =  [10, 10, 0, 10, 10, 10]
Torq_fault = np.matmul(CA_fault_3, u_input_fault)
print(f"\nIf PWM input is considering fault")
print(f"u1, u2, u3, u4, u5, u6: {u_input_fault}")
print(f"Torque and thrust: {Torq_fault}")


Control Allocation with FAULT in m3
If PWM input is NOT considering fault
u1, u2, u3, u4, u5, u6: [10, 10, 10, 10, 10, 10]
Torque and thrust: [-1.66666667  2.88683603  1.66666667  8.33333333]

If PWM input is considering fault
u1, u2, u3, u4, u5, u6: [10, 10, 0, 10, 10, 10]
Torque and thrust: [-1.66666667  2.88683603  1.66666667  8.33333333]
